In [1]:
%%capture
%pip install pandas
%pip install groq
%pip install transformers

In [2]:
import torch
import groq
from groq import Groq
from transformers import MarianMTModel, MarianTokenizer

import pandas as pd
import ast
from tqdm import tqdm
import time
import os

/home/fabiogr/miniconda3/envs/ml_proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize MarianMTModel

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [4]:
model_name = "Helsinki-NLP/opus-mt-tc-big-en-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(54776, 1024, padding_idx=54775)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(54776, 1024, padding_idx=54775)
      (embed_positions): MarianSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1

### Read dataset from CSV

In [5]:
df_dev = pd.read_csv('data/dev.csv')
df_train = pd.read_csv('data/train.csv')
df_tst = pd.read_csv('data/tst.csv')

In [6]:
df_dev.head()

,Unnamed: 0,context,question,answerA,answerB,answerC,correct
0,0,Tracy didn't go home that evening and resisted...,What does Tracy need to do before this?,make a new plan,Go home and see Riley,Find somewhere to go,C
1,1,Sydney walked past a homeless woman asking for...,How would you describe Sydney?,sympathetic,like a person who was unable to help,incredulous,A
2,2,Sasha protected the patients' rights by making...,What will patients want to do next?,write new laws,get petitions signed,live longer,B
3,3,Jordan was in charge of taking the food on the...,How would Jordan feel afterwards?,horrible that he let his friends down on the c...,happy that he doesn't need to do the cooking o...,very proud and accomplished about the camping ...,A
4,4,Kendall opened their mouth to speak and what c...,How would you describe Kendall?,a very quiet person,a very passive person,a very aggressive and talkative person,C


#### Test Run

In [7]:
row_index = 0  # select the row index you're interested in
columns_of_interest = ['context', 'question', 'answerA', 'answerB', 'answerC']  # select the columns you want

row_values = df_train.loc[row_index, columns_of_interest].tolist()

In [8]:
row_values

['Even though she had homework to do that night, Jesse helped Skylar study.',
 'What will Jesse want to do next?',
 'read homework to Skylar',
 'help Skylar finish',
 'skip her studying']

In [9]:
def translate(str_list, model):

    translated = []
    preamble_str = ">>por<< "
    str_list = [preamble_str + s for s in str_list]

    translated_tokens = model.generate(**tokenizer(str_list, return_tensors="pt", padding=True).to(device))
    for t in translated_tokens:
        translated.append(tokenizer.decode(t, skip_special_tokens=True))
    
    return translated

In [10]:
out = translate(row_values, model)

In [11]:
out

['Mesmo que ela tivesse trabalho de casa para fazer naquela noite, Jesse ajudou Skylar a estudar.',
 'O que Jesse vai querer fazer a seguir?',
 'Leia lição de casa para Skylar',
 'Ajuda Skylar finish',
 'ignorá-la estudando']

### Translate dev base first

In [12]:
# New dataframe for the translation
df_dev_pt = df_dev.copy()
df_dev_pt.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None

In [13]:
df_dev_pt

,Unnamed: 0,context,question,answerA,answerB,answerC,correct
0,0,None,None,None,None,None,C
1,1,None,None,None,None,None,A
2,2,None,None,None,None,None,B
3,3,None,None,None,None,None,A
4,4,None,None,None,None,None,C
...,...,...,...,...,...,...,...
1949,1949,None,None,None,None,None,A
1950,1950,None,None,None,None,None,B
1951,1951,None,None,None,None,None,C
1952,1952,None,None,None,None,None,B


In [14]:
def create_batch(df, step, base):
    row_values = []
    for i in range(step):
        columns_of_interest = ['context', 'question', 'answerA', 'answerB', 'answerC']  # select the columns you want
        row_values += (df.loc[base + i, columns_of_interest].tolist())
    return row_values

In [15]:
batch_size = 1
n_cols = 5

def translate_base(df, df_pt, model):
    for i in tqdm(range(0, len(df), batch_size)):
        
        strings = create_batch(df, batch_size, i)
        translated_strings = translate(strings, model)

        df_pt.iat[i, 1] = translated_strings[0]
        df_pt.iat[i, 2] = translated_strings[1]
        df_pt.iat[i, 3] = translated_strings[2]
        df_pt.iat[i, 4] = translated_strings[3]
        df_pt.iat[i, 5] = translated_strings[4]

In [16]:
translate_base(df_dev, df_dev_pt, model)

100%|██████████| 1954/1954 [08:40<00:00,  3.75it/s]


In [17]:
df_dev_pt

,Unnamed: 0,context,question,answerA,answerB,answerC,correct
0,0,Tracy não foi para casa naquela noite e resist...,O que Tracy precisa fazer antes disso?,Faça um novo plano,Vá para casa e veja Riley,Encontre um lugar para ir,C
1,1,Sydney passou por uma mulher sem-teto pedindo ...,Como você descreveria Sydney?,simpático,como uma pessoa que não foi capaz de ajudar,incrédulo,A
2,2,"Sasha protegeu os direitos dos pacientes, faze...",O que os pacientes vão querer fazer a seguir?,Escrever novas leis,obter petições assinadas,viver mais tempo,B
3,3,Jordan estava encarregado de levar a comida na...,Como Jordan se sentiria depois?,horrível que ele deixou seus amigos para baixo...,feliz que ele não precisa fazer o cozimento na...,muito orgulhoso e realizado sobre a viagem de ...,A
4,4,Kendall abriu a boca para falar e o que saiu c...,Como você descreveria Kendall?,uma pessoa muito tranquila,Uma pessoa muito passiva,uma pessoa muito agressiva e falante,C
...,...,...,...,...,...,...,...
1949,1949,Bailey destruiu o bunker do inimigo e ganhou a...,O que Bailey precisa fazer antes disso?,espionar os movimentos das tropas inimigas,Dê ao inimigo contra-inteligência,Vencer a guerra pelo seu país,A
1950,1950,Kai contou a Kendall a razão do acidente no la...,Como você descreveria Kai?,Mais à vontade,Como se tivessem tirado algo do peito,Sentindo-se triste,B
1951,1951,Kai melhorou a imagem de Jan e ela adorou como...,O que Jan vai querer fazer a seguir?,queria ser útil,queria que Jan fosse feliz,enquadrar a imagem,C
1952,1952,Os filhos da Tracy queriam gelado para que o A...,O que Aubrey precisa fazer antes disso?,Diga aos seus filhos para dizer obrigado,obter sorvete,Obrigado Aubrey,B


In [18]:
# Save to file
df_dev_pt.to_csv('translated/marian_mt/dev_pt.csv')

### Translate training and testing bases too

In [19]:
df_tst_pt = df_tst.copy()
df_tst_pt.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None
df_train_pt = df_train.copy()
df_train_pt.loc[:, ['context', 'question', 'answerA', 'answerB', 'answerC']] = None

translate_base(df_tst, df_tst_pt, model)
translate_base(df_train, df_train_pt, model)

100%|██████████| 33410/33410 [2:26:58<00:00,  3.79it/s]  


In [20]:
# Save to file
df_tst_pt.to_csv('translated/marian_mt/tst_pt.csv')
df_train_pt.to_csv('translated/marian_mt/train_pt.csv')